# **Fine-tuning Wav2Vec2 for English ASR with Transformers**

Wav2Vec2 is a pretrained model for Automatic Speech Recognition (ASR) and was released in September 2020 by Alexei Baevski, Michael Auli, and Alex Conneau.

Using a novel contrastive pretraining objective, Wav2Vec2 learns powerful speech representations from more than 50.000 hours of unlabeled speech, the model learns contextualized speech representations by randomly masking feature vectors before passing them to a transformer network.



For the first time, it has been shown that pretraining, followed by fine-tuning on very little labeled speech data achieves competitive results to state-of-the-art ASR systems. 

 we will fine-tune Wav2Vec2 without making use of a language model. It is much simpler to use Wav2Vec2 without a language model as an end-to-end ASR system and it has been shown that a standalone Wav2Vec2 acoustic model achieves impressive results. we fine-tune the "base"-sized on the rather small Timit dataset that contains just 5h of training data.

Wav2Vec2 is fine-tuned using Connectionist Temporal Classification (CTC), which is an algorithm that is used to train neural networks for sequence-to-sequence problems and mainly in Automatic Speech Recognition and handwriting recognition. 



In [ ]:

!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install soundfile
!pip install jiwer

  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-mx7cxf65
  Running command git clone -q https://github.com/huggingface/datasets.git /tmp/pip-req-build-mx7cxf65
     |████████████████████████████████| 245kB 13.6MB/s 
     |████████████████████████████████| 112kB 26.7MB/s 
  Created wheel for datasets: filename=datasets-1.5.0.dev0-cp37-none-any.whl size=189992 sha256=1a66e929f0f257d0e9a0f2b0fd21d5841c6e05a958b1d241abcc1d9f64b40b3b
  Stored in directory: /tmp/pip-ephem-wheel-cache-40dd_xhy/wheels/3e/af/ff/d1cdb5d0f9cff6eba2042a16b477ada497e23f1a3b6950b928
Successfully built datasets
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-29i1v63e
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-29i1v63e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 16.7MB/s 
     |█

## Prepare Data, Tokenizer, Feature Extractor

ASR models transcribe speech to text, which means that we both need a feature extractor that processes the speech signal to the model's input format  and a tokenizer that processes the model's output format to text. 

 the Wav2Vec2 model is thus accompanied by both a tokenizer, called Wav2Vec2CTCTokenizer and a feature extractor  called Wav2Vec2FeatureExtractor
we  start by creating the tokenizer responsible for decoding the model's predictions.

### Create Wav2Vec2CTCTokenizer

The pretrained Wav2Vec2 checkpoint maps the speech signal to a sequence of context representations . A fine-tuned Wav2Vec2 checkpoint needs to map this sequence of context representations to its corresponding transcription so that a linear layer has to be added on top of the transformer block. This linear layer is used to classifies each context representation to a token class 

The output size of this layer corresponds to the number of tokens in the vocabulary, which does **not** depend on Wav2Vec2's pretraining task, but only on the labeled dataset used for fine-tuning. So in the first step, we will take a look at Timit and define a vocabulary based on the dataset's transcriptions.

In [ ]:
from datasets import load_dataset, load_metric

timit = load_dataset("timit_asr")

Dataset timit_asr downloaded and prepared to /root/.cache/huggingface/datasets/timit_asr/clean/2.0.1/e7245f262bb418c1a5c67d67e00297322b1ad19eb65223676b790aa8003449e4. Subsequent calls will reuse this data.


In [ ]:
timit

DatasetDict({
    train: Dataset({
        features: ['file', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 1680
    })
})

 we will  consider the transcribed text for fine-tuning.



In [ ]:
timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])

we write  a short function to display some random samples of the dataset 

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(timit["train"].remove_columns(["file"]), num_examples=20)

,text
0,Everything in the final analysis reduced itself to sexual symbolism.
1,But it must be remembered that the plan should not be oriented geographically.
2,She had your dark suit in greasy wash water all year.
3,"Well, now we have two big theaters."
4,A screwdriver is made from vodka and orange juice.
5,"Mosquitoes exist in warm, humid climates."
6,Try to recall the events in chronological order.
7,Don't ask me to carry an oily rag like that.
8,A screwdriver is made from vodka and orange juice.
9,She had your dark suit in greasy wash water all year.


We can see that the transcriptions contain some special characters, such as `,.?!;:`. Without a language model, it is much harder to classify speech chunks to such special characters because they don't really correspond to a characteristic sound unit.
Also in order to understand the meaning of a speech signal, it is usually not necessary to include special characters in the transcription.

In addition, we normalize the text to only have lower case letters and append a word separator token at the end.

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [ ]:
timit = timit.map(remove_special_characters)

In [ ]:
show_random_elements(timit["train"].remove_columns(["file"]))

,text
0,the soft snow was deceitful underfoot
1,would a camera club be useful in taking pictures pertinent to plant safety
2,she had your dark suit in greasy wash water all year
3,don't ask me to carry an oily rag like that
4,coconut cream pie makes a nice dessert
5,don't ask me to carry an oily rag like that
6,don't ask me to carry an oily rag like that
7,she had your dark suit in greasy wash water all year
8,in every major cloverleaf traffic sometimes gets backed up
9,regular attendance is seldom required


.

In CTC, it is common to classify speech chunks into letters so we will  extract all distinct letters of the training and test data and build our vocabulary from this set of letters.

We write a mapping function that concatenates all transcriptions into one long transcription and then transforms the string into a set of chars. 


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
#batched=true to have access to all the transcription text at once 
vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])

In [ ]:
vocabs["train"]["all_text"]

In [ ]:
vocabs["train"]["vocab"]

[['m',
  'b',
  'o',
  'f',
  'y',
  'i',
  'a',
  'g',
  'u',
  'k',
  'w',
  't',
  'z',
  "'",
  'r',
  'h',
  's',
  'd',
  'n',
  'j',
  'c',
  'v',
  'p',
  'l',
  'q',
  ' ',
  'x',
  'e']]

In [ ]:
vocabs

DatasetDict({
    train: Dataset({
        features: ['all_text', 'vocab'],
        num_rows: 1
    })
    test: Dataset({
        features: ['all_text', 'vocab'],
        num_rows: 1
    })
})

Now, we create the union of all distinct letters in the training dataset and test dataset and convert the resulting list into an enumerated dictionary.

In [ ]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [ ]:
vocab_list

['m',
 'b',
 'o',
 'f',
 'y',
 'i',
 'a',
 'g',
 'u',
 'k',
 'w',
 't',
 'z',
 "'",
 'r',
 'h',
 's',
 'd',
 'n',
 'j',
 'c',
 'v',
 'p',
 'l',
 'q',
 ' ',
 'x',
 'e']

In [ ]:
len(vocab_list)

28

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 25,
 "'": 13,
 'a': 6,
 'b': 1,
 'c': 20,
 'd': 17,
 'e': 27,
 'f': 3,
 'g': 7,
 'h': 15,
 'i': 5,
 'j': 19,
 'k': 9,
 'l': 23,
 'm': 0,
 'n': 18,
 'o': 2,
 'p': 22,
 'q': 24,
 'r': 14,
 's': 16,
 't': 11,
 'u': 8,
 'v': 21,
 'w': 10,
 'x': 26,
 'y': 4,
 'z': 12}

 we also extracted the special characters `" "` and `'`.  we did not exclude those special characters because: 

- The model has to learn to predict when a word finished or else the model prediction would always be a sequence of chars which would make it impossible to separate words from each other.
- In English, we need to keep the `'` character to differentiate between words, `"it's"` and `"its"` which have very different meanings.

To make it clearer that `" "` has its own token class, we give it a more visible character `|`. In addition, we also add an "unknown" token so that the model can later deal with characters not encountered in Timit's training set. 
 we also add a padding token that corresponds to CTC's "blank token". The "blank token" is a core component of the CTC algorithm.

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict

{"'": 13,
 'a': 6,
 'b': 1,
 'c': 20,
 'd': 17,
 'e': 27,
 'f': 3,
 'g': 7,
 'h': 15,
 'i': 5,
 'j': 19,
 'k': 9,
 'l': 23,
 'm': 0,
 'n': 18,
 'o': 2,
 'p': 22,
 'q': 24,
 'r': 14,
 's': 16,
 't': 11,
 'u': 8,
 'v': 21,
 'w': 10,
 'x': 26,
 'y': 4,
 'z': 12,
 '|': 25}

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

30

 now our vocabulary is complete and consists of 30 tokens, which means that the linear layer that we will add on top of the pretrained Wav2Vec2 checkpoint will have an output dimension of 30.

Let's now save the vocabulary as a json file.

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In a final step, we use the json file to instantiate an object of the `Wav2Vec2CTCTokenizer` class.

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Next, we will create the feature extractor.

### Create Wav2Vec2 Feature Extractor

Speech is a continuous signal and to be treated by computers, it first has to be discretized, which is usually called **sampling**. The sampling rate hereby plays an important role in that it defines how many data points of the speech signal are measured per second. Therefore, sampling with a higher sampling rate results in a better approximation of the *real* speech signal but also necessitates more values per second.

**A pretrained checkpoint expects its input data to have been sampled more or less from the same distribution as the data it was trained on**. The same speech signals sampled at two different rates have a very different distribution, *e.g.*, doubling the sampling rate results in data points being twice as long. Thus, 
before fine-tuning a pretrained checkpoint of an ASR model, it is crucial to verify that the sampling rate of the data that was used to pretrain the model matches the sampling rate of the dataset used to fine-tune the model.

Wav2Vec2 was pretrained on the audio data of LibriSpeech and LibriVox which both were sampling with 16kHz. Our fine-tuning dataset, Timit was  also sampled with 16kHz. If the fine-tuning dataset would have been sampled with a rate lower or higher than 16kHz, we first would have had to up or downsample the speech signal to match the sampling rate of the data used for pretraining. 



In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

the feature extractor and tokenizer are *wrapped* into a single `Wav2Vec2Processor` class so that one only needs a `model` and `processor` object.

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
 from google.colab import drive
 drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
 processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-base-timit-demo")

Next, we can prepare the dataset.

### Preprocess Data

`Wav2Vec2` expects the audio file in the format of a 1-dimensional array, so in the first step, let's load all audio files into the dataset object.

Let's first check the serialization format of the downloaded audio files by looking at the first training sample.

In [ ]:
timit["train"][0]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/404950a46da14eac65eb4e2a8317b1372fb3971d980d91d5d5b221275b1fd7e0/data/TRAIN/DR4/MMDM0/SI681.WAV',
 'text': 'would such an act of refusal be useful '}


An audio file usually stores both its values and the sampling rate with which the speech signal was digitalized. We want to store both in the dataset and write a `map(...)` function accordingly.

In [ ]:
import soundfile as sf

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = sf.read(batch["file"])
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

In [ ]:
timit = timit.map(speech_file_to_array_fn, remove_columns=timit.column_names["train"], num_proc=4)

we listen to a couple of audio files to better understand the dataset and verify that the audio was correctly loaded. 



In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(timit["train"]))

ipd.Audio(data=np.asarray(timit["train"][rand_int]["speech"]), autoplay=True, rate=16000)

In [ ]:
rand_int = random.randint(0, len(timit["train"]))

print("Target text:", timit["train"][rand_int]["target_text"])
print("Input array shape:", np.asarray(timit["train"][rand_int]["speech"]).shape)
print("Sampling rate:", timit["train"][rand_int]["sampling_rate"])

Target text: when peeling an orange it is hard not to spray juice 
Input array shape: (53760,)
Sampling rate: 16000


Finally, we can process the dataset to the format expected by the model for training. We will again make use of the `map(...)` function.

First, we check that all data samples have the same sampling rate (of 16kHz).
Second, we extract the `input_values` from the loaded audio file. In our case, this includes only normalization, but for other speech models, this step could correspond to extracting, *e.g.* Log-Mel features
Third, we encode the transcriptions to label ids.


In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
timit_prepared = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], batch_size=8, num_proc=4, batched=True)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is de

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is de

In [ ]:
timit_prepared

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 1680
    })
})

In [ ]:
timit_prepared["train"]["input_values"][0]

In [ ]:
timit_prepared["train"]["labels"][0]

[10,
 2,
 8,
 23,
 17,
 25,
 16,
 8,
 20,
 15,
 25,
 6,
 18,
 25,
 6,
 20,
 11,
 25,
 2,
 3,
 25,
 14,
 27,
 3,
 8,
 16,
 6,
 23,
 25,
 1,
 27,
 25,
 8,
 16,
 27,
 3,
 8,
 23,
 25]

In [ ]:
timit_prepared["train"][0]

## Training & Evaluation

we will use une classe predefinie called trainer for training from transformers library 

- data collator. In contrast to most NLP models, Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning Wav2Vec2 requires a special padding data collator

### Set-up Trainer

 in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them . This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [ ]:
#A DataCollator is a function that takes a list of samples from a Dataset and collate them into a batch, as a dictionary. of Tensors.
#defines the padding , it will dynamically pad the inputs received 
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
   

    processor: Wav2Vec2Processor
    #pad to the longest sequence in the batch 
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

 the predominant metric in ASR is the word error rate (WER)

In [ ]:
wer_metric = load_metric("wer")

The model will return a sequence of logit vectors:
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.

A logit vector $\mathbf{y}_1$ contains the log-odds for each word in the vocabulary we defined earlier(probabilité de chaque lettre de notre vocab en timestep i), thus $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`. We are interested in the most likely prediction of the model and thus take the `argmax(...)` of the logits. Also, we transform the encoded labels back to the original string by replacing `-100` with the `pad_token_id` and decoding the ids 

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Now, we can load the pretrained `Wav2Vec2` checkpoint. The tokenizer's `pad_token_id` must be to define the model's `pad_token_id` or in the case of `Wav2Vec2ForCTC` also CTC's blank token. To save GPU memory, we enable PyTorch's gradient checkpointing and also set the loss reduction to "*mean*".

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The first component of Wav2Vec2 consists of a stack of CNN layers(7 layers ) that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretrainind and as stated in the  does not need to be fine-tuned anymore. 
so we freeze those layers

In [ ]:
model.freeze_feature_extractor()

In a final step, we define all parameters related to training. 

- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model




In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
   output_dir="/content/gdrive/MyDrive/wav2vec2-base-timit-demo",
  
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

Now, all instances can be passed to Trainer

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit_prepared["train"],
    eval_dataset=timit_prepared["test"],
    tokenizer=processor.feature_extractor,
)

### Training

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
500,3.770400,2.934055,1.000000,108.421900,15.495000
1000,0.907100,0.407282,0.406175,111.068200,15.126000
1500,0.191600,0.375646,0.363655,109.682100,15.317000
2000,0.110700,0.374007,0.328027,108.647700,15.463000
2500,0.074900,0.416617,0.331886,108.472300,15.488000
3000,0.057400,0.428464,0.322238,109.946900,15.280000
3500,0.046400,0.400957,0.308731,109.096700,15.399000
4000,0.034300,0.425326,0.306595,109.004000,15.412000


Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
500,3.770400,2.934055,1.000000,108.421900,15.495000
1000,0.907100,0.407282,0.406175,111.068200,15.126000
1500,0.191600,0.375646,0.363655,109.682100,15.317000
2000,0.110700,0.374007,0.328027,108.647700,15.463000
2500,0.074900,0.416617,0.331886,108.472300,15.488000
3000,0.057400,0.428464,0.322238,109.946900,15.280000
3500,0.046400,0.400957,0.308731,109.096700,15.399000
4000,0.034300,0.425326,0.306595,109.004000,15.412000


TrainOutput(global_step=4350, training_loss=0.5991971930142107, metrics={'train_runtime': 10514.5688, 'train_samples_per_second': 0.414, 'total_flos': 3.9858324576556134e+18, 'epoch': 30.0, 'init_mem_cpu_alloc_delta': 340118, 'init_mem_gpu_alloc_delta': 377847808, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1220413, 'train_mem_gpu_alloc_delta': 1112347136, 'train_mem_cpu_peaked_delta': 127309620, 'train_mem_gpu_peaked_delta': 4097821696})

The final WER  0.3

### Evaluate

In the final part, we evaluate our fine-tuned model on the test set 
we load the `processor` and `model`.

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("/content/gdrive/MyDrive/wav2vec2-base-timit-demo")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("/content/gdrive/MyDrive/wav2vec2-base-timit-demo")

Now, we will make use of the `map(...)` function to predict the transcription of every test sample and to save the prediction in the dataset itself. We will call the resulting dictionary `"results"`. 


In [ ]:
def map_to_result(batch):
  model.to("cuda")
  input_values = processor(
      batch["speech"], 
      sampling_rate=batch["sampling_rate"], 
      return_tensors="pt"
  ).input_values.to("cuda")

  with torch.no_grad():
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  
  return batch

In [ ]:
results = timit["test"].map(map_to_result)

Let's compute the overall WER now.

In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["target_text"])))

Test WER: 0.195




Let's take a look at some predictions to see what errors are made by the model.

In [ ]:
show_random_elements(results.remove_columns(["speech", "sampling_rate"]))

,pred_str,target_text
0,aim to balence your employ you benefit package,aim to balance your employee benefit package
1,the fog prevented them from ariving on toime,the fog prevented them from arriving on time
2,young children should avoid exposure to contageous diseases,young children should avoid exposure to contagious diseases
3,artificial intelligence is for real,artificial intelligence is for real
4,their props were two step ladters a chair and a poulm fan,their props were two stepladders a chair and a palm fan
5,if people were more generous there would be no need for wealfair,if people were more generous there would be no need for welfare
6,the fish began to lep frantically on the surface of the small eak,the fish began to leap frantically on the surface of the small lake
7,her righte hand agus whonever the barametric pressure changes,her right hand aches whenever the barometric pressure changes
8,only loawyers loved miliuneers,only lawyers love millionaires
9,the nearest cynagob may not be within walk in distance,the nearest synagogue may not be within walking distance
